# Runs 

<div style="text-align:center">
    <img src="./images/mlflow_run.jpeg" alt="MLFlow Run" />
</div>


A MLflow run is a unit of work in MLflow that represents the execution of a machine learning experiment or a piece of code. It tracks the parameters, metrics, artifacts, and metadata associated with the run. MLflow runs allow you to log and track experiments, compare different runs, and reproduce results. Each run is associated with an experiment and can have multiple tags, parameters, metrics, and artifacts.

## Creating a MLflow Run

### Using start_run

```python
mlflow.start_run()
```

In [1]:
import mlflow 
from mlflow_for_ml_dev.utils.utils import get_root_project

In [2]:
# Set the tracking uri to the mlruns folder in the root project
# This will allow us to store run metadata in the mlruns folder in the root project

artifact_location = get_root_project() / "runs" / "mlruns"

mlflow.set_tracking_uri(artifact_location.as_uri())
print(artifact_location)

C:\Users\manue\projects\mlflow_for_ml_dev\runs\mlruns


## Creating MLflow experiment

In [3]:
# Define the experiment name and tags
experiment_name = "runs-01"
tags = {"project_name":"UNDEFINED", "topic":"run_management"}

# Create an experiment
experiment_id = mlflow.create_experiment(name=experiment_name, tags=tags)

# Set the experiment. This will allow us to log runs to the experiment
# It returns an experiment object
experiment = mlflow.set_experiment(experiment_name)

## Starting an Active Run.

In [4]:
run = mlflow.start_run()

In [5]:
# print the type of the run object
type(run)

mlflow.tracking.fluent.ActiveRun

In [6]:
run.info.to_proto()

run_id: "5ac5be251aed48b485e838b89863676d"
run_uuid: "5ac5be251aed48b485e838b89863676d"
run_name: "rare-gnu-461"
experiment_id: "594575729744005488"
user_id: "manue"
status: RUNNING
start_time: 1721091394319
artifact_uri: "file:///C:/Users/manue/projects/mlflow_for_ml_dev/runs/mlruns/594575729744005488/5ac5be251aed48b485e838b89863676d/artifacts"
lifecycle_stage: "active"

In [7]:
run.data.to_dictionary()

{'metrics': {},
 'params': {},
 'tags': {'mlflow.runName': 'rare-gnu-461',
  'mlflow.source.name': 'c:\\Users\\manue\\projects\\mlflow_for_ml_dev\\.venv\\Lib\\site-packages\\ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'manue'}}

In [8]:
# Simulating a machine Learning run

# Machine learing code here
# ...

# logging some random parameters
mlflow.log_param("param1", 5)
mlflow.log_param("param2", 5)
mlflow.log_param("param3", 5)

# logging some random metrics
mlflow.log_metric("metric1", 15)
mlflow.log_metric("metric2", 52)
mlflow.log_metric("metric3", 35)

In [9]:
# Get the updated run object. This method will return the updated run object
run = mlflow.get_run(run_id=run.info.run_id)

In [10]:
run.data.to_dictionary()

{'metrics': {'metric1': 15.0, 'metric2': 52.0, 'metric3': 35.0},
 'params': {'param1': '5', 'param2': '5', 'param3': '5'},
 'tags': {'mlflow.runName': 'rare-gnu-461',
  'mlflow.source.name': 'c:\\Users\\manue\\projects\\mlflow_for_ml_dev\\.venv\\Lib\\site-packages\\ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'manue'}}

In [11]:
#starting a new run without ending the previous one will throw an error
run2 = mlflow.start_run()

Exception: Run with UUID 5ac5be251aed48b485e838b89863676d is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [12]:
# To start a new run, first end the current run with mlflow.end_run().
mlflow.end_run()

In [13]:
#starting a new run
run2 = mlflow.start_run()

In [14]:
mlflow.end_run()

## Providing more run metadata 

In [15]:
# creating run providing more info. Such as run name, description and tags.
run_tags = {"tag1": "value1", "tag2": "value2"}

run3 = mlflow.start_run(
    run_name="run_with_tags",
    tags=run_tags,
    description="This is a run with tags"
)

In [16]:
# ending the run
mlflow.end_run()

#### Using with statement


> About `mlflow.start_run()`
> 
> The return value of `mlflow.start_run()` can be used as a context manager within a `with` block. Otherwise, you must call `end_run()` to terminate the current run.

Example:

```python
with mlflow.start_run() as run:
    print("Log metrics and params")
```

In [17]:
mlflow.end_run() # end the active run to start a new one
with mlflow.start_run(run_name="Run 2", experiment_id=experiment.experiment_id) as run:

    # Your ML code here
    # ...
    
    active_run = mlflow.active_run()
    print(type(active_run))  
    print("Active Run: ", run.info.run_id)
    print("Active Run: ", active_run.info.run_id)
    print("\n \n")


# outside the with block
active_run = mlflow.active_run()
print(type(active_run))    

<class 'mlflow.tracking.fluent.ActiveRun'>
Active Run:  003df1628d2f4667b8ac40ffd97fc629
Active Run:  003df1628d2f4667b8ac40ffd97fc629

 

<class 'NoneType'>


In [18]:
# active_run is None because the run has ended.
active_run == None

True

## Using MlflowClient

In [20]:
# creating a mlflow client
client = mlflow.MlflowClient()

In [21]:
# This will create a run with the specified name and tags
created_run = client.create_run(
    experiment_id=experiment.experiment_id,
    run_name="test_run",
    tags = {
        "tag1": "value1",
        "tag2": "value2"
        }
    )

In [22]:
# Notice that the run is of type mlflow.entities.Run. Before it was of type mlflow.entities.ActiveRun
type(created_run)

mlflow.entities.run.Run

In [23]:
# since the client creates a Run object, there will be no active run. (ActiveRun is a different object)
run = mlflow.active_run()
type(run)

NoneType

In [24]:
# Loging to MLflow without an active run will create a new run automatically. 
# Some mlflow functions create a run automatically if there is no active run.
mlflow.log_param("param1", 5)
run = mlflow.active_run()
type(run)

mlflow.tracking.fluent.ActiveRun

In [25]:
# Random name created by mlflow
run.info.run_name

'rogue-ant-685'

In [26]:
# End the active run
mlflow.end_run()

In [27]:
# We can resume the run by providing the run_id and then log into it 
with mlflow.start_run(run_id = created_run.info.run_id):
    mlflow.log_param("param1", 5)
    run = mlflow.active_run()
    print(run.info.run_name)

# This will also end the run

test_run


In [28]:
# Creating a second run with the mlflow client
created_run = client.create_run(
    experiment_id=experiment.experiment_id,
    run_name="test_run",
    tags = {
        "tag1": "value1",
        "tag2": "value2"
    }
)

In [29]:
# There are some logging functions that allows to log into a specific run by providing the run_id
mlflow.log_metric(key="metric1", value=5, run_id=created_run.info.run_id)

In [31]:
# Although, not all logging functions have the run_id parameter.
# For example, mlflow.log_param() does not have the run_id parameter
mlflow.log_param("param1", 5)

5

In [32]:
# ending the active run
mlflow.end_run()

In [33]:
# Instead of using the "with" block, we can use the client to terminate the run.
client.set_terminated(run_id=created_run.info.run_id)

## Updating Run Tags

In [35]:
# end the active run to start a new one
mlflow.end_run()
with mlflow.start_run(run_name="Run 3", experiment_id=experiment.experiment_id) as run:
    # set a single tag
    mlflow.set_tag("tag3", "value3")
    
    # Set multiple tags as a dictionary
    mlflow.set_tags({"tag4": "value4", "tag5": "value5"})

## Update run description

In [36]:
# we can update the description of the run 
# In this case we are updating the description of the run with the name "Run 4"
with mlflow.start_run(run_name="Run 4", experiment_id=experiment.experiment_id) as run:
    
    #Update description
    # the tag "mlflow.note.content" is used to store the description of the run
    mlflow.set_tag("mlflow.note.content", "This is a new description")

In [37]:
# including markdown.
with mlflow.start_run(run_name="Run 5", experiment_id=experiment.experiment_id) as run:
    #Update description
    mlflow.set_tag("mlflow.note.content", "# This is a new description")

    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 15)


## Retrieve run information

In [38]:
run.info.run_name

'Run 5'

In [39]:
run = mlflow.get_run(run_id = run.info.run_id)

In [40]:
run.info.to_proto()

run_id: "1addb617ee694576b415fb3883047820"
run_uuid: "1addb617ee694576b415fb3883047820"
run_name: "Run 5"
experiment_id: "594575729744005488"
user_id: "manue"
status: FINISHED
start_time: 1721093472082
end_time: 1721093472143
artifact_uri: "file:///C:/Users/manue/projects/mlflow_for_ml_dev/runs/mlruns/594575729744005488/1addb617ee694576b415fb3883047820/artifacts"
lifecycle_stage: "active"

In [42]:
run.data.to_dictionary()

{'metrics': {'metric1': 15.0},
 'params': {'param1': '5'},
 'tags': {'mlflow.note.content': '# This is a new description',
  'mlflow.runName': 'Run 5',
  'mlflow.source.name': 'c:\\Users\\manue\\projects\\mlflow_for_ml_dev\\.venv\\Lib\\site-packages\\ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'manue'}}